In [14]:
# !pip install openai python-aiconfig

In [15]:
# import openai
# import aiconfig
# from google.colab import userdata
# openai.api_key = userdata.get('OPENAI_API_KEY')


In [16]:
# Non-Colab
import json
import openai
openai.api_key = open("/home/jacobjensen/secrets/openai_api_key.txt", "r").read().strip()

## Function Calling

In [17]:
from pydantic import create_model
from typing import Callable, Dict, List
import inspect, json
from inspect import Parameter
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [18]:
def inject_function_call_spec(functions: List[Callable]):
    spec = json.load(open("function_calling_list_builder_aiconfig.json", 'r'))
    spec["prompts"][0]["metadata"]["model"]["settings"]["functions"] = [schema(f) for f in functions]
    return spec



In [19]:
async def initialize_list(items: List[str]) -> List[str]:
    """returns an initial list of items"""
    return items

async def add_to_list(li: List[str], items: List[str]) -> List[str]:
    """returns a list with items added, does not add duplicates"""
    return li + [item for item in items if item not in set(li)]

async def remove_from_list(li: List[str], items: List[str]) -> List[str]:
    """returns a list with items removed"""
    return [item for item in li if item not in set(items)]

# Note - this function isn't about calling an API or mutating state, but about enforcing an output format for assessments
async def assess_list(li: List[str], assessment: Dict[str, str]) -> List[str]: 
    """Accepts a dictionary of assessments of a list, keyed by the name of characteristic being assessed."""
    return assessment


In [20]:
import copy
import json
import uuid
from aiconfig import AIConfigRuntime
from aiconfig import PromptInput
from aiconfig.model_parser import InferenceOptions
from pprint import pprint


async def function_calling(function_list):
    function_call_spec = inject_function_call_spec(function_list)
    function_dict = {f.__name__: f for f in function_list}
    config = AIConfigRuntime.create(**function_call_spec)
    return config, function_dict

async def call_function(function_call, function_dict):
    args = function_call.get("arguments", None)
    args = json.loads(args) if args else None

    if not args:
        raise Exception("No arguments found")

    function = function_dict.get(function_call["name"], None)
    if not function:
        raise Exception(f"Function {function_call['name']} not found")
    else:
        return await function(**args)



In [21]:
config, function_dict = await function_calling([initialize_list, add_to_list, remove_from_list, assess_list])

/tmp/ipykernel_496394/3314632738.py:11: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [22]:
user_input = "I want to buy supplies for a 15 mile hike in hot weather."
li = []
promptToRun = "recommendList"
inference_options = InferenceOptions(stream=True)

async def run_chat(user_input, li, promptToRun=promptToRun, inference_options=inference_options):
    params = {
        "user_input": user_input,
        "list": li,
    }
    output = await config.run(promptToRun, params, inference_options)
    return output


await config.resolve(prompt_name="recommendList", params={"user_input": user_input, "list": li})

{'functions': [{'name': 'initialize_list',
   'description': 'returns an initial list of items',
   'parameters': {'properties': {'items': {'items': {'type': 'string'},
      'title': 'Items',
      'type': 'array'}},
    'required': ['items'],
    'title': 'Input for `initialize_list`',
    'type': 'object'}},
  {'name': 'add_to_list',
   'description': 'returns a list with items added, does not add duplicates',
   'parameters': {'properties': {'li': {'items': {'type': 'string'},
      'title': 'Li',
      'type': 'array'},
     'items': {'items': {'type': 'string'},
      'title': 'Items',
      'type': 'array'}},
    'required': ['li', 'items'],
    'title': 'Input for `add_to_list`',
    'type': 'object'}},
  {'name': 'remove_from_list',
   'description': 'returns a list with items removed',
   'parameters': {'properties': {'li': {'items': {'type': 'string'},
      'title': 'Li',
      'type': 'array'},
     'items': {'items': {'type': 'string'},
      'title': 'Items',
      'type

In [23]:

while True:
    model_output = await run_chat(user_input, li, promptToRun, inference_options)

    output = model_output[0] if isinstance(model_output, list) else model_output
    print(output)

    if output.output_type == "error":
        raise ValueError(f"Error during inference: {output.ename}: {output.evalue}")

    message = output.data

    # If there is no function call, we're done and can exit this loop
    if not message.get("function_call", None):
        print("No function call found")
        break

    # If there is a function call, we generate a new message with the role 'function'.

    result = await call_function(message.get("function_call"), function_dict)
    print(result)

    new_message = {
        "role": "function",
        "name": message["function_call"]["name"],
        "content": json.dumps(result),
    }

    promptToRun = f"functionCallResult-{uuid.uuid4()}"

    existing_prompt = config.get_prompt("recommendList")

    new_prompt = copy.deepcopy(existing_prompt)
    new_prompt.name = promptToRun
    new_prompt.input = PromptInput(**new_message)
    new_prompt.outputs = []

    config.add_prompt(new_prompt.name, new_prompt)

    print(f"{new_message}\n")

output_type='execute_result' execution_count=0 data={'function_call': {'arguments': '{\n  "items": ["water", "snacks", "sunscreen", "hat", "sunglasses", "hiking boots"]\n}', 'name': 'initialize_list'}, 'role': 'assistant'} mime_type=None metadata={'finish_reason': 'function_call'}
['water', 'snacks', 'sunscreen', 'hat', 'sunglasses', 'hiking boots']
{'role': 'function', 'name': 'initialize_list', 'content': '["water", "snacks", "sunscreen", "hat", "sunglasses", "hiking boots"]'}

output_type='execute_result' execution_count=0 data={'function_call': {'arguments': '{\n  "li": ["water", "snacks", "sunscreen", "hat", "sunglasses", "hiking boots"],\n  "items": ["backpack", "map", "compass", "first aid kit", "extra clothing"]\n}', 'name': 'add_to_list'}, 'role': 'assistant'} mime_type=None metadata={'finish_reason': 'function_call'}
['water', 'snacks', 'sunscreen', 'hat', 'sunglasses', 'hiking boots', 'backpack', 'map', 'compass', 'first aid kit', 'extra clothing']
{'role': 'function', 'name

TypeError: assess_list() missing 1 required positional argument: 'assessment'

In [24]:
user_input = "Thanks! can you trim this list to just 4 items?"
li = result
await config.resolve(prompt_name="recommendList", params={"user_input": user_input, "list": li})


{'functions': [{'name': 'initialize_list',
   'description': 'returns an initial list of items',
   'parameters': {'properties': {'items': {'items': {'type': 'string'},
      'title': 'Items',
      'type': 'array'}},
    'required': ['items'],
    'title': 'Input for `initialize_list`',
    'type': 'object'}},
  {'name': 'add_to_list',
   'description': 'returns a list with items added, does not add duplicates',
   'parameters': {'properties': {'li': {'items': {'type': 'string'},
      'title': 'Li',
      'type': 'array'},
     'items': {'items': {'type': 'string'},
      'title': 'Items',
      'type': 'array'}},
    'required': ['li', 'items'],
    'title': 'Input for `add_to_list`',
    'type': 'object'}},
  {'name': 'remove_from_list',
   'description': 'returns a list with items removed',
   'parameters': {'properties': {'li': {'items': {'type': 'string'},
      'title': 'Li',
      'type': 'array'},
     'items': {'items': {'type': 'string'},
      'title': 'Items',
      'type

In [ ]:

while True:
    model_output = await run_chat(user_input, li, promptToRun, inference_options)

    output = model_output[0] if isinstance(model_output, list) else model_output
    print(output)

    if output.output_type == "error":
        raise ValueError(f"Error during inference: {output.ename}: {output.evalue}")

    message = output.data

    # If there is no function call, we're done and can exit this loop
    if not message.get("function_call", None):
        print("No function call found")
        break

    # If there is a function call, we generate a new message with the role 'function'.

    result = await call_function(message.get("function_call"), function_dict)

    new_message = {
        "role": "function",
        "name": message["function_call"]["name"],
        "content": json.dumps(result),
    }

    promptToRun = f"functionCallResult-{uuid.uuid4()}"

    existing_prompt = config.get_prompt("recommendList")

    new_prompt = copy.deepcopy(existing_prompt)
    new_prompt.name = promptToRun
    new_prompt.input = PromptInput(**new_message)
    new_prompt.outputs = []

    config.add_prompt(new_prompt.name, new_prompt)

    print(f"{new_message}\n")

In [ ]:
config.save("list_builder.json")